#### Dashboard:

In [ ]:
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import dash_auth
import json

# Carregar dados de autenticação de um arquivo JSON:
with open(r'C:\Users\diego\OneDrive\Área de Trabalho\Programação\Hashtag Programação\Material Aulas Python Impressionador\autenticacao_dash.json', 'r') as arquivo_json:
    dados_autenticacao = json.load(arquivo_json)

# Acessar a variável USUARIOS dos dados de autenticação:
USUARIOS = dados_autenticacao['USUARIOS']

#Criando o aplicativo Dash:
app = Dash(__name__)

#Solicitando a autenticação:
auth = dash_auth.BasicAuth(app, USUARIOS)

#Importando o banco de dados:
df = pd.read_excel(r'Arquivo Mentoria Dash\Vendas.xlsx')

#Criando os gráficos:
fig = px.bar(df, x='Produto', y='Quantidade', color='ID Loja', barmode='group')
fig2 = px.scatter(df, x='Quantidade', y='Valor Final', color='Produto', size='Valor Unitário', size_max=60)

#Criando lista de marcas:
lista_marcas = list(df['Marca'].unique())
lista_marcas.append('Todas')

# Criando o Layout da página do Dash:
app.layout = html.Div(children=[
    html.H1(children='Meu Dashboard'),

    html.H2(children='Dashboard de Vendas em Python'),
    
    html.H3(children='Vendas de cada Produto por Loja', id='subtitulo'),
    
    dcc.RadioItems(lista_marcas, value='Todas', id='selecao_marcas'),
    
    dcc.Graph(id='vendas_por_loja', figure=fig),
    dcc.Graph(id='distribuicao_vendas', figure=fig2),
], style={'text-align': 'center'})

#Callback:
@app.callback(
    Output('subtitulo', 'children'),
    Output('vendas_por_loja', 'figure'),
    Output('distribuicao_vendas', 'figure'),
    Input('selecao_marcas', 'value'),
)
def selecionar_marca(marca):
    if marca == 'Todas':
        texto = 'Vendas de cada Produto por Loja'
        fig = px.bar(df, x='Produto', y='Quantidade', color='ID Loja', barmode='group')
        fig2 = px.scatter(df, x='Quantidade', y='Valor Final', color='Produto', size='Valor Unitário', size_max=60)
    else:
        df_filtrada = df.loc[df['Marca']==marca, :]
        texto = f'Vendas de cada Produto por Loja da Marca {marca}'
        fig = px.bar(df_filtrada, x='Produto', y='Quantidade', color='ID Loja', barmode='group')
        fig2 = px.scatter(df_filtrada, x='Quantidade', y='Valor Final', color='Produto', size='Valor Unitário', size_max=60)
    return texto, fig, fig2

# Colocando o site(dashboard) no ar:
if __name__ == '__main__':
    app.run_server(debug=False)